<a href="https://colab.research.google.com/github/cnrgrl/PANDAS/blob/main/05_Boolean_Selection_Single_Conditions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# uncomment the following line, if you are using google collab
!rm -r Pandas
!git clone https://github.com/Wuebbelt/Pandas.git

Cloning into 'Pandas'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 77 (delta 12), reused 75 (delta 10), pack-reused 0
Unpacking objects: 100% (77/77), done.


# Boolean Selection Single Conditions

Boolean Selection, also referred to as **Boolean Indexing**, is the process of selecting subsets of rows from DataFrames (or Series) based on the actual data values and NOT by their labels or integer locations. All of the previous subset selections were done using either labels or integer location. Those selections had nothing to do with the actual values.

### Examples of Boolean Selection

Let's see some examples of actual questions (in plain English) that boolean selection can help us answer from the bikes dataset. The term **query** is used to refer to these sorts of questions.

* Find all rides by males
* Find all rides with duration longer than 2 hours
* Find all rides that took place between March and June of 2015.
* Find all the rides with a duration longer than 2 hours by females with temperature higher than 90 degrees

### All queries have a logical condition

Each of the above queries have a strict logical condition that must be checked one row at a time.

### Keep or discard an entire row of data

If you were to manually answer the above queries, you would need to scan each row and determine whether the row, as a whole, meets the condition. If so, then it is kept in the result, otherwise it is discarded.

### Each row will have a True or False value associated with it

When you perform boolean selection, each row of the DataFrame (or value of a Series) has a `True` or `False` value associated with it that corresponds to the outcome of the logical condition.

### Begin with a small DataFrame

Let's perform our first boolean selection on our sample DataFrame. Let's read it in now.

In [ ]:
import pandas as pd
df = pd.read_csv('Pandas/sample_data.csv', index_col=0)
df

,state,color,food,age,height,score
name,,,,,,
Jane,NY,blue,Steak,30,165,4.6
Niko,TX,green,Lamb,2,70,8.3
Aaron,FL,red,Mango,12,120,9.0
Penelope,AL,white,Apple,4,80,3.3
Dean,AK,gray,Cheese,32,180,1.8
Christina,TX,black,Melon,33,172,9.5
Cornelia,TX,red,Beans,69,150,2.2


## Manually filtering the data

Let's find all the people who are younger than 30 years old. We will do this manually by inspecting the data.

### Create a list of booleans

By inspecting the data, we see that `Niko`, `Aaron`, and `Penelope` are all under 30 years old. To signify which people are under 30, we create a list of 7 boolean values corresponding to the 7 rows in the DataFrame. The values in the list that correspond with the positions of `Niko`, `Aaron`, and `Penelope` are `True`. All other values are `False`. `Niko`, `Aaron`, and `Penelope` are the 2nd, 3rd, and 4th rows, so these are the locations in the list that are `True`.

In [ ]:
filt = [False, True, True, True, False, False, False]

### Variable name `filt`

The variable name `filt` will be used throughout the book to refer to the sequence of booleans. You are free to use any variable name you like for the sequence of booleans, but being consistent makes your code easier to understand. I chose `filt` because it short for the word 'filter'. Boolean selection filters the data for a particular condition, which is why this variable name makes sense to me.

### Place this list in just the brackets

The above list has `True` in the 2nd, 3rd, and 4th positions. These will be the rows that are kept in the result during boolean selection. Place the list inside *just the brackets* to complete the selection.

In [ ]:
df[filt]

,state,color,food,age,height,score
name,,,,,,
Niko,TX,green,Lamb,2,70,8.3
Aaron,FL,red,Mango,12,120,9.0
Penelope,AL,white,Apple,4,80,3.3


### Wait a second… Isn’t `[ ]` just for column selection?

The primary purpose of *just the brackets* for a DataFrame is to select one or more columns by using either a string or a list of strings. All of a sudden, this example shows entire rows being selected with boolean values. This is what makes pandas, unfortunately, a confusing library to learn and use.

## Operator Overloading
*Just the brackets* is **overloaded**. Depending on the inputs, pandas will do something completely different. Here are the rules for the different objects passed to *just the brackets*.

* **string** — return a column as a Series
* **list of strings** — return all those column names as a DataFrame
* **sequence of booleans** — select all rows where `True`

In summary, *just the brackets* primarily selects columns, but if you pass it a sequence of booleans it will select all rows that are `True`.

### Using booleans in a Series and not a list

Instead of a list, we can use a Series of booleans, which will make the same selection. Below, we use the Series constructor to create a Series object. The Series must have the same index as the DataFrame it is selecting from in order to work properly, so we create it with the same index as the original DataFrame. This automatic alignment of the index is important and covered in the **Joining Data** part.

In [ ]:
filt = pd.Series([False, True, True, True, False, False, False], 
                 index=df.index)
filt

name
Jane         False
Niko          True
Aaron         True
Penelope      True
Dean         False
Christina    False
Cornelia     False
dtype: bool

### Use the boolean Series to do the boolean selection

Placing the Series directly in the brackets selects only the rows that correspond with the `True` values in the Series.

In [ ]:
df[filt]

,state,color,food,age,height,score
name,,,,,,
Niko,TX,green,Lamb,2,70,8.3
Aaron,FL,red,Mango,12,120,9.0
Penelope,AL,white,Apple,4,80,3.3


## Practical boolean selection

We almost never create boolean lists/Series manually like we did above, but instead use the actual data to create them.

### Creating boolean Series from column data

By far the most common way to create a boolean Series is from the values of one particular column. We test a condition using one of the six comparison operators:

* `<`
* `<=`
* `>`
* `>=`
* `==`
* `!=`

Let's begin doing practical boolean selection examples by reading in the bikes dataset.

In [ ]:
bikes = pd.read_csv('Pandas/bikes.csv')
bikes.head(3)

,trip_id,usertype,gender,starttime,stoptime,tripduration,from_station_name,latitude_start,longitude_start,dpcapacity_start,to_station_name,latitude_end,longitude_end,dpcapacity_end,temperature,visibility,wind_speed,precipitation,events
0,7147,Subscriber,Male,2013-06-28 19:01:00,2013-06-28 19:17:00,993,Lake Shore Dr & Monroe St,41.881050,-87.616970,11.0,Michigan Ave & Oak St,41.90096,-87.623777,15.0,73.9,10.0,12.7,-9999.0,mostlycloudy
1,7524,Subscriber,Male,2013-06-28 22:53:00,2013-06-28 23:03:00,623,Clinton St & Washington Blvd,41.883380,-87.641170,31.0,Wells St & Walton St,41.89993,-87.634430,19.0,69.1,10.0,6.9,-9999.0,partlycloudy
2,10927,Subscriber,Male,2013-06-30 14:43:00,2013-06-30 15:01:00,1040,Sheffield Ave & Kingsbury St,41.909592,-87.653497,15.0,Dearborn St & Monroe St,41.88132,-87.629521,23.0,73.0,10.0,16.1,-9999.0,mostlycloudy


### Create a boolean Series

Let's create a boolean Series by determining which rows have a trip duration greater than 1,000 seconds. To make the comparison, we select the `tripduration` column as a Series and compare it against the integer 1,000.

In [ ]:
filt = bikes['tripduration'] > 1000
filt.head(3)

0    False
1    False
2     True
Name: tripduration, dtype: bool

When we write `bikes['tripduration'] > 1000`, pandas compares each value in the `tripduration` column against 1000. It returns a new Series the same length as `tripduartion` with boolean values corresponding to the outcome of the comparison. Let's verify that the `filt` Series is the same length as the DataFrame.

In [ ]:
len(filt)

50089

In [ ]:
len(bikes)

50089

### Manually verify correctness

Take a look at the `tripduration` column to manually verify that only the third row satisfied the condition. That ride lasted 1,040 seconds which is greater than 1,000 so it's value in the resulting boolean Series is `True`. The first two rides lasted less than 1,000 seconds and thus resulted as `False`.

### Complete the boolean selection

We can now place the `filt` boolean Series into *just the brackets* to filter the entire DataFrame to return all the rows in the bikes dataset that have a trip duration greater than 1,000. Manually verify that the visible `tripduration` values are greater than 1,000.

In [ ]:
bikes[filt].head(3)

,trip_id,usertype,gender,starttime,stoptime,tripduration,from_station_name,latitude_start,longitude_start,dpcapacity_start,to_station_name,latitude_end,longitude_end,dpcapacity_end,temperature,visibility,wind_speed,precipitation,events
2,10927,Subscriber,Male,2013-06-30 14:43:00,2013-06-30 15:01:00,1040,Sheffield Ave & Kingsbury St,41.909592,-87.653497,15.0,Dearborn St & Monroe St,41.881320,-87.629521,23.0,73.0,10.0,16.1,-9999.0,mostlycloudy
8,21028,Subscriber,Male,2013-07-03 15:21:00,2013-07-03 15:42:00,1300,Clinton St & Washington Blvd,41.883380,-87.641170,31.0,Wood St & Division St,41.903320,-87.672730,15.0,71.1,8.0,0.0,-9999.0,cloudy
10,24383,Subscriber,Male,2013-07-04 17:17:00,2013-07-04 17:42:00,1523,Morgan St & 18th St,41.858086,-87.651073,15.0,Damen Ave & Pierce Ave,41.909396,-87.677692,19.0,79.0,10.0,9.2,-9999.0,mostlycloudy


### How many rows have a trip duration greater than 1000?
To answer this question, let's assign the result of the boolean selection to a variable and then compare the number of rows between it and the original DataFrame.

In [ ]:
bikes_duration_1000 = bikes[filt]

Let's find the number of rows in each DataFrame.

In [ ]:
len(bikes)

50089

In [ ]:
len(bikes_duration_1000)

10178

We compute that 20% of the rides are longer than 1,000 seconds.

In [ ]:
len(bikes_duration_1000) / len(bikes)

0.20319830701351593

## Boolean selection in one line
Often, you will see boolean selection completed in a single line of code instead of the two lines we used above. The expression for the filter is placed directly inside the brackets. Although this method will save a line of code, I recommend assigning the filter as a separate variable to help with readability.

In [ ]:
bikes[bikes['tripduration'] > 1000].head(3)

,trip_id,usertype,gender,starttime,stoptime,tripduration,from_station_name,latitude_start,longitude_start,dpcapacity_start,to_station_name,latitude_end,longitude_end,dpcapacity_end,temperature,visibility,wind_speed,precipitation,events
2,10927,Subscriber,Male,2013-06-30 14:43:00,2013-06-30 15:01:00,1040,Sheffield Ave & Kingsbury St,41.909592,-87.653497,15.0,Dearborn St & Monroe St,41.881320,-87.629521,23.0,73.0,10.0,16.1,-9999.0,mostlycloudy
8,21028,Subscriber,Male,2013-07-03 15:21:00,2013-07-03 15:42:00,1300,Clinton St & Washington Blvd,41.883380,-87.641170,31.0,Wood St & Division St,41.903320,-87.672730,15.0,71.1,8.0,0.0,-9999.0,cloudy
10,24383,Subscriber,Male,2013-07-04 17:17:00,2013-07-04 17:42:00,1523,Morgan St & 18th St,41.858086,-87.651073,15.0,Damen Ave & Pierce Ave,41.909396,-87.677692,19.0,79.0,10.0,9.2,-9999.0,mostlycloudy


## Single condition expression

Our first example tested a single condition (whether the trip duration was 1,000 or more). Let's test a different single condition and find all the rides that happened when the weather was cloudy. We use the `==` operator to test for equality and again pass this variable to the brackets which completes our selection.

In [ ]:
filt = bikes['events'] == 'cloudy'
bikes[filt].head(3)

,trip_id,usertype,gender,starttime,stoptime,tripduration,from_station_name,latitude_start,longitude_start,dpcapacity_start,to_station_name,latitude_end,longitude_end,dpcapacity_end,temperature,visibility,wind_speed,precipitation,events
6,18880,Subscriber,Male,2013-07-02 17:47:00,2013-07-02 17:56:00,565,Clark St & Randolph St,41.884576,-87.631890,31.0,Ravenswood Ave & Irving Park Rd,41.954690,-87.673930,19.0,66.0,10.0,15.0,-9999.0,cloudy
7,19689,Subscriber,Male,2013-07-03 09:07:00,2013-07-03 09:16:00,505,State St & Van Buren St,41.877181,-87.627844,27.0,Franklin St & Jackson Blvd,41.877708,-87.635321,27.0,64.0,7.0,5.8,-9999.0,cloudy
8,21028,Subscriber,Male,2013-07-03 15:21:00,2013-07-03 15:42:00,1300,Clinton St & Washington Blvd,41.883380,-87.641170,31.0,Wood St & Division St,41.903320,-87.672730,15.0,71.1,8.0,0.0,-9999.0,cloudy


## Summary of single condition boolean selection

Boolean selection refers to the act of filtering your data based on the values and not on the labels or integer location. There are two main steps to do boolean selection:

1. Create a boolean Series - commonly done by comparing one column of data to another value
2. Place the boolean Series inside *just the brackets* to filter the data

## Exercises

Continue to use the bikes dataset for the first few exercises.

### Exercise 1

<span  style="color:green; font-size:16px">Find all the rides with temperature below 0.</span>

### Exercise 2

<span  style="color:green; font-size:16px">Find all the rides with wind speed greater than 30.</span>

### Exercise 3

<span  style="color:green; font-size:16px">Find all the rides that began from station Millennium Park.</span>

### Exercise 4

<span  style="color:green; font-size:16px">Find all the rides with wind speed less than 0. How is this possible?</span>

### Exercise 5

<span  style="color:green; font-size:16px">Find all the rides where the starting number of bikes at the station (dpcapacity_start) was more than 50.</span>

### Exercise 6

<span  style="color:green; font-size:16px">Did any rides happen in temperature over 100 degrees.</span>

Read in the movie dataset by executing the cell below and use it for the following exercises.

In [ ]:
import pandas as pd
movie = pd.read_csv('Pandas/movie.csv', index_col='title')
movie.head(3)

,year,color,content_rating,duration,director_name,director_fb,actor1,actor1_fb,actor2,actor2_fb,actor3,actor3_fb,gross,genres,num_reviews,num_voted_users,plot_keywords,language,country,budget,imdb_score
title,,,,,,,,,,,,,,,,,,,,,
Avatar,2009.0,Color,PG-13,178.0,James Cameron,0.0,CCH Pounder,1000.0,Joel David Moore,936.0,Wes Studi,855.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,723.0,886204,avatar|future|marine|native|paraplegic,English,USA,237000000.0,7.9
Pirates of the Caribbean: At World's End,2007.0,Color,PG-13,169.0,Gore Verbinski,563.0,Johnny Depp,40000.0,Orlando Bloom,5000.0,Jack Davenport,1000.0,309404152.0,Action|Adventure|Fantasy,302.0,471220,goddess|marriage ceremony|marriage proposal|pi...,English,USA,300000000.0,7.1
Spectre,2015.0,Color,PG-13,148.0,Sam Mendes,0.0,Christoph Waltz,11000.0,Rory Kinnear,393.0,Stephanie Sigman,161.0,200074175.0,Action|Adventure|Thriller,602.0,275868,bomb|espionage|sequel|spy|terrorist,English,UK,245000000.0,6.8


### Exercise 7

<span  style="color:green; font-size:16px">Select all movies that have Tom Hanks as `actor1`. How many of these movies has he starred in?</span>

### Exercise 8

<span  style="color:green; font-size:16px">Select movies with an IMDB score greater than 9.</span>

### Exercise 9

<span  style="color:green; font-size:16px">Write a function that accepts a single parameter to find the number of movies for a given content rating. Use the function to find the number of movies for ratings 'R', 'PG-13', and 'PG'.</span>